# Operações controladas

In [1]:
import qutip
import numpy as np
import qiskit

In [2]:
def xgate():
    mat = np.matrix([[0, 1], [1, 0]])
    return qutip.Qobj(mat, dims = [[2],[2]])
X = xgate()

**Teorema**. Suponha que $U$ é um operador unitário sobre um qbit. Então existem operadores unitários $A$, $B$, $C$ sobre um único qbit, tais que $ABC = I$ e $e^{i\alpha}AXBXC = U$.

Prova. Faça $A = R_z(\beta)R_y(\gamma/2)$, $B=R_y(-\gamma/2)R_z(-(\delta+\beta)/2)$ e $C = R_z((\delta-\beta)/2)$. Onde $\alpha$, $\beta$, $\gamma$ e $\delta$ são definidos em [operações sobre um qbit](operadores1qbit.ipynb)

In [3]:
# Exemplo Hadamard

alpha = -np.pi/2
beta = np.pi
gamma = 3*np.pi/2
delta = 0
A = qutip.rz(beta) * qutip.ry(gamma/2)
B = qutip.ry(-gamma/2) * qutip.rz(-(delta + beta)/2)
C = qutip.rz((delta-beta)/2)

In [4]:
A * B * C

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[1. 0.]
 [0. 1.]]

In [5]:
mat = np.matrix([[0, 1], [1, 0]])
np.e**(1j*alpha) * A * X * B * X * C 

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[ 0.70710678  0.70710678]
 [ 0.70710678 -0.70710678]]

### Implementando Hadamard controlado no qiskit

In [6]:
def contrlGate(circuito, alpha, beta, gamma, delta, q0, q1):


    # aplicar C
    circuito.rz((delta-beta)/2, q1)

    # cnot
    circuito.cx(q0,q1)

    # aplicar B
    circuito.rz(-(delta + beta)/2, q1)
    circuito.ry(-gamma/2., q1)

    # cnot
    circuito.cx(q0,q1)

    # aplicar A
    circuito.ry(gamma/2., q1)
    circuito.rz(beta, q1)
    
    # e^{i \alpha}
    circuito.u1(alpha, q0)



In [7]:
# Hadamard controlado com estado inicial |11>
alpha = -np.pi/2.
beta = np.pi
gamma = 3.*np.pi/2
delta = 0.
q = qiskit.QuantumRegister(2)
cr = qiskit.ClassicalRegister(1)
circuito = qiskit.QuantumCircuit(q, cr)

circuito.x(q)
contrlGate(circuito, alpha, beta, gamma, delta, q[0], q[1])
circuito.measure(q[1], cr[0])
circuito.draw()




In [8]:
from qiskit import BasicAer
backend = BasicAer.get_backend('qasm_simulator')
job = qiskit.execute(circuito, backend, shots=1024)
result = job.result()
counts = result.get_counts(circuito)
print(counts)

{'1': 526, '0': 498}


In [9]:
# Hadamard controlado com estado inicial |00>
alpha = -np.pi/2.
beta = np.pi
gamma = 3.*np.pi/2
delta = 0.
q = qiskit.QuantumRegister(2)
cr = qiskit.ClassicalRegister(1)
circuito = qiskit.QuantumCircuit(q, cr)

#circuito.x(q)
contrlGate(circuito, alpha, beta, gamma, delta, q[0], q[1])
circuito.measure(q[1], cr[0])
circuito.draw()

from qiskit import BasicAer
backend = BasicAer.get_backend('qasm_simulator')
job = qiskit.execute(circuito, backend, shots=1024)
result = job.result()
counts = result.get_counts(circuito)
print(counts)

{'0': 1024}


## Z controlado

In [10]:
# Circuito Z controlado 
alpha = np.pi/2
beta = 0
gamma = np.pi
delta = np.pi
q = qiskit.QuantumRegister(2)
cr = qiskit.ClassicalRegister(1)
circuito = qiskit.QuantumCircuit(q, cr)

#circuito.x(q)
contrlGate(circuito, alpha, beta, gamma, delta, q[0], q[1])
circuito.measure(q[1], cr[0])
circuito.draw()

Alguns operadores podem ser removidos do circuito para implementar o operador Z-controlado. 

In [11]:
qutip.ry(0)

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[1. 0.]
 [0. 1.]]

In [12]:
qutip.rz(0)

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[1. 0.]
 [0. 1.]]

Outra possibilidade para implementar Z-controlado

In [13]:
q = qiskit.QuantumRegister(2)
circuito = qiskit.QuantumCircuit(q)
circuito.h(q[0])
circuito.cx(q[0], q[1])
circuito.h(q[0])
circuito.draw()

In [14]:
 def raiz_operador(operador):
        eig = np.linalg.eig(operador)
        aval0 = eig[0][0]
        aval1 = eig[0][1]
        avec0 = eig[1][:,0]
        avec1 = eig[1][:,1] 
        raiz = np.sqrt(aval0 + 0j) * avec0 * avec0.T + np.sqrt(aval1 + 0j) * avec1 * avec1.T
        return raiz

In [15]:
X = np.matrix([[0, 1],[1, 0]])
H = (1/np.sqrt(2)) *np.matrix([[1, 1],[1, -1]])
V = raiz_operador(X)
print(V)
print('-----')
print(V*V)
V*V.conjugate().T

[[0.5+0.5j 0.5-0.5j]
 [0.5-0.5j 0.5+0.5j]]
-----
[[0.-2.46519033e-32j 1.+0.00000000e+00j]
 [1.+0.00000000e+00j 0.+2.46519033e-32j]]


matrix([[1.00000000e+00+0.j, 2.46519033e-32+0.j],
        [2.46519033e-32+0.j, 1.00000000e+00+0.j]])

In [16]:
norma = np.linalg.norm(V[0, 0])

In [17]:
alpha = V[0, 0] / norma

In [18]:
V_l = (1/alpha) * V

In [19]:
V_l

matrix([[0.70710678+0.j        , 0.        -0.70710678j],
        [0.        -0.70710678j, 0.70710678+0.j        ]])

In [20]:
theta = np.arccos(V_l[0,0])
theta

(0.7853981633974485-0j)